# Ritvik_Tests:
# Question: Which types of incidents have the most undersends/oversends? (Answered but needs editing)
# Question: Can we visualize when during the week is the highest traffic of incidents? (Answered)
# Question: Can we measure the distribution of workload amongst medics? (Answered)
# Question: Can we determine which geographical areas are most EMS active? (In Progress)

## Imports

In [ ]:
%matplotlib inline
import pandas as pd
import operator
import psycopg2
import numpy as np
import datetime
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
from scipy.stats.stats import pearsonr
import matplotlib.lines as mlines
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from statsmodels.tsa import stattools
import statsmodels.api as sm
import scipy
import random
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab

mpl.rcdefaults()

# Prettier plots:
#import matplotlib
#matplotlib.pyplot.style.use('ggplot') # Not supported by current version
pd.options.display.mpl_style = 'default'  # Will raise error because deprecated

## Connection to database

In [ ]:
# Read database parameters from default_profile
dbitems = {}
with open('default_profile') as f:
    for line in f.readlines():
        item = line.split(" ")[1].split("=")
        dbitems[item[0]] = item[1].strip()

In [ ]:
# Connect to database with psycopg2
try:
    conn = psycopg2.connect("dbname='%s' user='%s' host='%s' password='%s'"%(dbitems['PGDATABASE'],dbitems['PGUSER'],dbitems['PGHOST'],dbitems['PGPASSWORD']))
except:
    print "Unable to connect to the database"

In [ ]:
# Connect to database with sqalchemy
conn_sqlalch = create_engine('postgresql+psycopg2://%s:%s@%s/%s'%(dbitems['PGUSER'],dbitems['PGPASSWORD'],dbitems['PGHOST'],dbitems['PGDATABASE']))

# Function Bank

In [ ]:
def convert_date_to_datetime(s):
    """ Convert string datetime from the CAD db to datetime object
    
    :param string
    :return datetime object
    
    """
    s=str(s)
    s= s.replace('-',' ').replace(':',' ').split()
    s = map(int,s)
    d = datetime.datetime(*s)
    return d

In [ ]:
def intdate_to_datetime(int_date):
    s = str(int(int_date))
    year = int(s[0:4])
    month = int(s[4:6])
    day = int(s[6:8])
    hour = int(s[8:10])
    minute = int(s[10:12])
    second = int(s[12:14])
    
    return datetime.datetime(year, month, day, hour, minute, second)
    

In [ ]:
def cast_to_datetime(df, date_col_name):
    """ Find the datetime col in a dataframe and converts it to datetime object 
    using the function 'convert_date_to_datetime'
        
    :param dataframe df : has a col with dates  
    :param string date_col_name : the name of the date col
    :rtype dataframe
    :return the updated dataframe
    """

     
    df[date_col_name] = df[date_col_name].apply(convert_date_to_datetime)
    return df
    

In [ ]:
def gen_late_dispatch_inc(thresh_time, date_col_name, id_col_name):
    """ Get events where units were dispatched after thresh_time
        
    :param int thresh_time  
    :param string id_col_name name of the column
    :param string date_col_name name of the date column
    :return dataframe containing incidents with late dispatch
    """
    return pd.read_sql_query("select distinct " + id_col_name + ",count(*) from clean_csvs.dbo_rfirehouseapparatus join clean.events_ using (" + id_col_name \
                             + ") where " + date_col_name + "::timestamp > min_dispatch_time + interval'"+str(thresh_time)\
                             +"'second group by 1;;",conn) 
    


In [ ]:
def gen_dict_from_cols(col_name1, col_name2):
    """ Generate an indexed dataframe from two columns 
    
    :param string col_name1, col_name2 : column names to fetch from the db
    :rtype dateframe
    :return df indexed dataframe 
    """
    
    df = pd.read_sql_query("select {},{} from raw.cad".format(col_name1, col_name2),conn) 
    
    id_to_type = {}
    gb = df.groupby(col_name1)
    for name,group in gb:
        id_to_type[name] = group[col_name2].iloc[0]
    return id_to_type
    

In [ ]:
def gen_number_late_inc_by_type(excess_units, id_to_type, type_name):
    """ Generate dictionary with key as type of incident and value as 
        number of times units were dispatched late
    
    :param list excess_units : incident numbers where late units were dispatched
    :param dictionary id_to_type : dictionary of incident numbers to type of incident
    :param string type_name : name of incident type column
    :rtype dictionary
    :return excess_punit dictionary of units sent late by type of incident
    """
    
    excess_punit = {}
    
    for item in excess_units:
        if id_to_type[item] in excess_punit.keys():
            excess_punit[id_to_type[item]] += 1
        else:
            excess_punit[id_to_type[item]] = 1
                
    return excess_punit
        

In [ ]:
def gen_pct_distr(type_to_id):
    
    """ Generate dictionary with key as type of incident and value as 
        percent of times units were dispatched late
    
    :param dictionary type_to_id : dictionary of type_of_incident to incident number
    :rtype dictionary
    :return pct_distr dictionary of percentages of late incidents for each incident type
    """
    
    late_pct_inc = {}
    full_len = 0
    for key in type_to_id.keys():
        full_len += type_to_id[key]
    for key in type_to_id.keys():
        late_pct_inc[key] = float(type_to_id[key])/full_len
    return late_pct_inc

In [ ]:
def num_strip(s):
    """ Strip a string of numbers
    
    :param string s : string
    :return input string with no numbers
    """
    return s.strip('0123456789')

In [ ]:
def gen_nontransports(gb, date_col_name, hospital_time_name, unit_id_name, id_to_type):
    nontransports = {}
    for id_type in id_to_type.values():
        nontransports[id_type] = 0
    for name,group in gb:
        units = group[unit_id_name].apply(num_strip)
        transports = ['MT','M']
        both = [i for i in units if i in transports]
        if len(both) > 0:
            if all(v is None for v in group[hospital_time_name]):
                nontransports[id_to_type[name]] += 1
    return nontransports
        
        
        

In [ ]:
def gen_time_tuple(t):
    
    """ Input a datetime object and generate its attributes and store in a list of tuples
    
    :param datetime object t 
    :rtype list of tuples
    :return list of tuples with first element a double of day of week and hour and second element a triple
    with day, month, and year
    """
    
    day_of_week = t.weekday()
    hour = t.hour
    day = t.day
    month = t.month
    year = t.year
    week_of_year = t.isocalendar()[1]
    return [day_of_week,hour,day,month,year, (week_of_year, year)]

In [ ]:
def requires_transport(val):
    
    if val == None:
        return 0
    return 1

In [ ]:
def is_Medic(m):
    return m[0] == 'M'

In [ ]:
def drop_time_nan(df, date_col_name):
    df = df.dropna(subset=[date_col_name]) # drop the N/A values
    return df

In [ ]:
def get_year(t):
    return t.year

In [ ]:
def dispatch_tag(t):
    return (t,'d')
def clear_tag(t):
    return (t,'c')

In [ ]:
def assign_shift_number(t_tup, first_shift):
    return (t_tup[0] - first_shift).days + 1

In [ ]:
def running_mean(x, N):
    return np.convolve(x, np.ones((N,))/N)[(N-1):]

In [ ]:
def ymd(t):
    return (t.year, t.month, t.day)

In [ ]:
def binary_date(d, goal):
    return d[0].year == goal.year and d[0].month == goal.month and d[0].day == goal.day

# Code

In [ ]:
cad_df = pd.read_sql_query("SELECT i_eventnumber, iiu_tclear, iiu_tdispatch, pun_unitid, iiu_tenroutehospital FROM clean_csvs.dbo_rfirehouseapparatus", conn)
inc_type_map = pd.read_sql_query("SELECT i_eventnumber, i_ktypeinfo FROM clean_csvs.dbo_iincident",conn)
code_to_type = pd.read_sql_query("SELECT iti_typeinfo_pk, iti_typeid FROM clean_csvs.udt4_itypeinfo", conn)
#cad_df = pd.read_sql_query("SELECT i_eventnumber, iiu_tdispatch, iti_typeid, pun_unitid, iiu_tenroutehospital FROM raw.cad", conn)

In [ ]:
cad_df = pd.merge(cad_df, inc_type_map, on = 'i_eventnumber')

In [ ]:
cad_df = cad_df.rename(columns = {'i_ktypeinfo': 'type_code'})

In [ ]:
code_to_type = code_to_type.rename(columns = {'iti_typeinfo_pk': 'type_code'})

In [ ]:
cad_df = pd.merge(cad_df, code_to_type, on = 'type_code')

In [ ]:
cad_df = cad_df.drop('type_code', 1)

In [ ]:
#cad_df = cast_to_datetime(cad_df, 'iiu_tdispatch')
cad_df = drop_time_nan(cad_df, 'iiu_tdispatch')

In [ ]:
cad_df['transport'] = cad_df['iiu_tenroutehospital'].apply(requires_transport)

In [ ]:
cad_df['full_unit_id'] = cad_df['pun_unitid']

In [ ]:
cad_df['pun_unitid'] = cad_df['pun_unitid'].apply(num_strip)

In [ ]:
cad_df['is_Medic'] = cad_df['pun_unitid'].apply(is_Medic)

In [ ]:
excess_units = gen_late_dispatch_inc(60, 'iiu_tdispatch', 'i_eventnumber').set_index('i_eventnumber')

In [ ]:
id_to_type = cad_df.groupby(by=['i_eventnumber'])['iti_typeid'].first()

In [ ]:
transports = cad_df.groupby(by=['i_eventnumber'])['transport'].sum()

In [ ]:
medic_sent = cad_df.groupby(by=['i_eventnumber'])['is_Medic'].sum()

In [ ]:
event_feat_df = pd.concat([id_to_type, transports, medic_sent], axis=1)

In [ ]:
event_feat_df = event_feat_df.join(excess_units)

In [ ]:
event_feat_df = event_feat_df.fillna(0)

In [ ]:
gb_type = event_feat_df.groupby('iti_typeid')

In [ ]:
excess_punit = {}
type_to_id = {}
non_transports = {}
for name,group in gb_type:
    excess_punit[name] = np.count_nonzero(group['count'])
    type_to_id[name] = len(group)
    true_false = (group['transport'] < group['is_Medic']).value_counts()
    if True in true_false:
        non_transports[name] = true_false.ix[True]
    else:
        non_transports[name] = 0

In [ ]:
pct_undersend = {}
pct_oversend = {}

In [ ]:
for key in [i for i in type_to_id.keys() if i in excess_punit.keys()]:
    pct_undersend[key] = float(excess_punit[key])/type_to_id[key]
for key in [i for i in type_to_id.keys() if i in non_transports.keys()]:
    pct_oversend[key] = float(non_transports[key])/type_to_id[key]
    

In [ ]:
sorted_dict = sorted(pct_undersend.items(), key=operator.itemgetter(1))


In [ ]:
pct_distr = gen_pct_distr(type_to_id)

In [ ]:
sorted_freq_list = [i[0] for i in sorted(pct_distr.items(), key=operator.itemgetter(1))]

In [ ]:
sorted_freq_list.reverse()

In [ ]:
top_ten = sorted_freq_list[0:10]

In [ ]:
common_pct_us = {}
common_pct_os = {}
for i_type in top_ten:
    common_pct_us[i_type] = pct_undersend[i_type] 
    common_pct_os[i_type] = pct_oversend[i_type]

In [ ]:
plt.bar(range(len(common_pct_us)), common_pct_us.values(),  align='center' )
plt.xticks(range(len(common_pct_us)), common_pct_us.keys(), rotation = 'vertical')
plt.xlim(-1,10)
plt.title("Undersent Resources Fractions for Top Ten Incident Types")

In [ ]:
plt.bar(range(len(common_pct_os)), common_pct_os.values(),  align='center' )
plt.xticks(range(len(common_pct_os)), common_pct_os.keys(), rotation = 'vertical')
plt.xlim(-1,10)
plt.title("Undersent Resources Fractions for Top Ten Incident Types")

In [ ]:

font0 = FontProperties()
font = font0.copy()

ax = plt.subplot(111)
mpl.rcParams['font.family'] = 'monospace'
font.set_family(font)
ax.bar(np.arange(-.5,19.5,2), [i*100 for i in common_pct_us.values()],width=0.5,color='b',align='center', label = 'Percent Undersent')
ax.bar(np.arange(0,20,2), [i*100 for i in common_pct_os.values()],width=0.5,color='r',align='center', label = 'Percent Oversent')
ax.set_xticklabels(common_pct_os.keys(), rotation = 'vertical')
ax.set_xticks(np.arange(-.25,19.75,2))
ax.set_xlim(-1,19)
ax.set_xlabel('Incident Type')
ax.set_ylabel('Percent of Over/Under Sending (%)')
ax.set_title('Percent of Over and Under Sending by Incident Type')

blue_line = mlines.Line2D([], [], color='blue', marker='*',
                          markersize=15, label='Blue stars')
legend = plt.legend(loc='upper right', shadow=True, fontsize='x-large')
plt.gcf().subplots_adjust(bottom=0.25)
fig = mpl.pyplot.gcf()
mpl.rcParams.update({'font.size': 16})
fig.set_size_inches(12,7)
plt.savefig('Under_Over_Sending.jpg')

# HeatMap

In [ ]:
heat_df = pd.DataFrame()

medic_df = cad_df[cad_df['pun_unitid'] == 'M']

time_data = medic_df['iiu_tdispatch'].apply(gen_time_tuple)

heat_df['ID'] = medic_df['i_eventnumber']
heat_df['day_of_week'] = [i[0] for i in time_data]
heat_df['hour'] = [i[1] for i in time_data]
heat_df['day'] = [i[2] for i in time_data]
heat_df['month'] = [i[3] for i in time_data]
heat_df['year'] = [i[4] for i in time_data]
heat_df['week_year_id'] = [i[5] for i in time_data]

day_hour_gb = heat_df.groupby(['day_of_week', 'hour'])
month_day_gb = heat_df.groupby(['month', 'day'])

In [ ]:
hm_dayhour = np.zeros((7,24))

for name,group in day_hour_gb:
    sub_gb = group.groupby('week_year_id')
    lens = []
    for name2,group2 in sub_gb:
        lens.append(len(group2))
    med = np.mean(lens)
    hm_dayhour[name[0]][name[1]] = med
    


In [ ]:
fig, ax = plt.subplots()

mpl.rcParams['font.family'] = 'monospace'
mpl.rcdefaults()
cax = ax.imshow(hm_dayhour, interpolation='nearest', cmap=cm.coolwarm)
#ax.set_xlabel('Hour')
#ax.set_ylabel('Day')
ax.set_xlim(-0.5,23.5)
times = ['12AM', '1AM', '2AM', '3AM', '4AM', '5AM', '6AM', '7AM', '8AM', '9AM', '10AM', '11AM', \
         '12PM', '1PM', '2PM', '3PM', '4PM', '5PM', '6PM', '7PM', '8PM', '9PM', '10PM', '11PM']
ax.set_xticklabels(times, fontsize = 20, rotation = 90)
ax.set_yticklabels(["Monday","Tuesday","Wednesday",'Thursday','Friday','Saturday','Sunday'], fontsize = 20)
ax.set_xticks(np.arange(-0.5,23.5,1))
ax.set_yticks(np.arange(0,7,1))
ax.grid(b=False)



#ax.set_title("Median Medic Units Sent by Hour of Day")

# Add colorbar, make sure to specify tick locations to match desired ticklabels
cbar = fig.colorbar(cax, ticks=[-1, 0, 1],orientation='horizontal')
fig.set_size_inches(18.5, 10.5)
cbar.set_ticks([np.min(hm_dayhour), np.mean(hm_dayhour), np.max(hm_dayhour)])

cbar.ax.tick_params(labelsize=20) 

fig.savefig('Hour_Day_HMAP.pdf')

# Day of Year Heatmap

In [ ]:
hm_monthday = np.zeros((12,31))

for name,group in month_day_gb:
    sub_gb = group.groupby('year')
    lens = []
    for name2,group2 in sub_gb:
        lens.append(len(group2))
    med = np.mean(lens)
    hm_monthday[name[0]-1][name[1]-1] = med
hm_monthday[hm_monthday == 0] = np.nan

In [ ]:
fig, ax = plt.subplots()
mpl.rcdefaults()
cax = ax.imshow(hm_monthday, interpolation='nearest', cmap=cm.coolwarm)
#ax.set_xlabel('Day')
#ax.set_ylabel('Month')
ax.set_xlim(-0.5,30.5)
ax.set_xticklabels(range(1,32), fontsize = 20)
ax.set_yticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], fontsize = 20)
ax.set_xticks(np.arange(0,31,1))
ax.set_yticks(np.arange(0,12,1))
ax.grid(b=False)

#ax.set_title("Median Medic Units Sent by Day of Month")




# Add colorbar, make sure to specify tick locations to match desired ticklabels
cbar = fig.colorbar(cax, ticks=[-1, 0, 1], orientation='horizontal')
cbar.set_ticks([np.nanmin(hm_monthday), np.nanmean(hm_monthday), np.nanmax(hm_monthday)])
cbar.ax.tick_params(labelsize=20) 
fig.set_size_inches(18.5, 10.5)
fig.savefig('Day_Month_HMAP.pdf')

# Which Medic is Most in Demand?

In [ ]:
p_unit = cad_df[cad_df['full_unit_id'].apply(num_strip) == 'M']['full_unit_id']

In [ ]:
medic_dict= {}
for i in p_unit:
    if i in medic_dict.keys():
        medic_dict[i] += 1
    else:
        medic_dict[i] = 1

In [ ]:
medic_tups = [(v, k) for v,k in medic_dict.iteritems()]


In [ ]:
medic_tups = sorted(medic_tups, key=lambda x: x[1])

In [ ]:
medic_tups.reverse()

In [ ]:
m_thresh = 0.1*max([i[1] for i in medic_tups])

In [ ]:
medic_tups = [i for i in medic_tups if i[1] > m_thresh]

In [ ]:
tot = sum([i[1] for i in medic_tups])

In [ ]:
pct_tups = []
for i in range(len(medic_tups)):
    pct_tups.append((medic_tups[i][0], medic_tups[i][1] / float(tot)*100))

In [ ]:
fair_level = 1.0/12*100

In [ ]:
plt.bar(range(len(pct_tups)), [i[1] for i in pct_tups],  align='center',color = 'red')
plt.xticks(range(len(pct_tups)), [i[0] for i in pct_tups], rotation = 'vertical')
plt.xlim(-0.5,11.5)
plt.title("Distribution of Medic Demand")
plt.xlabel('Medic')
plt.ylabel('Percent of Workload (%)')
line1 = plt.axhline(y=fair_level, label = "Fair Workload Level")

legend = plt.legend(loc='upper right', shadow=True, fontsize='large')

plt.gcf().subplots_adjust(bottom=0.25)
fig = mpl.pyplot.gcf()

plt.savefig('MedicDemand.jpg')

# Idle time by medic unit by year

In [ ]:
year_of_interest = 2015

In [ ]:
year_df = pd.read_sql_query("SELECT i_eventnumber,iiu_tdispatch, pun_unitid, iiu_tclear,iiu_tenroutehospital FROM clean_csvs.dbo_rfirehouseapparatus WHERE DATE_PART('year', iiu_tdispatch) = " + str(year_of_interest) + ";", conn)

In [ ]:
year_df['isTransport'] = year_df['iiu_tenroutehospital'].apply(requires_transport)
year_df = year_df.drop('iiu_tenroutehospital', 1)

In [ ]:
year_df['unit_type'] = year_df['pun_unitid'].apply(num_strip)

In [ ]:
year_medic_df = year_df[year_df['unit_type'] == 'M']

In [ ]:
year_medic_df = year_medic_df.drop('unit_type',1)

In [ ]:
#year_medic_df['iiu_tdispatch_id'] = year_medic_df['iiu_tdispatch'].apply(dispatch_tag)
#year_medic_df['iiu_tclear_id'] = year_medic_df['iiu_tclear'].apply(clear_tag)


year_medic_df['iiu_tdispatch_id'] = zip(year_medic_df['iiu_tdispatch'], ['d']*len(year_medic_df), year_medic_df['pun_unitid'])
year_medic_df['iiu_tclear_id'] = zip(year_medic_df['iiu_tclear'], ['c']*len(year_medic_df), year_medic_df['pun_unitid'], year_medic_df['isTransport'])

In [ ]:
year_medic_df = year_medic_df.sort('iiu_tdispatch')
year_medic_df = year_medic_df.drop('iiu_tdispatch',1)
year_medic_df = year_medic_df.drop('iiu_tclear',1)

In [ ]:
first_shift = datetime.datetime(year_of_interest, 1, 1, 7, 0, 0 )

In [ ]:
year_medic_df['shift_num'] = year_medic_df['iiu_tdispatch_id'].apply(assign_shift_number, args = (first_shift,))

In [ ]:
year_medic_df = year_medic_df[year_medic_df['shift_num'] > 0]

In [ ]:
gb_medic = year_medic_df.groupby('pun_unitid')

In [ ]:
idle_times = {}

In [ ]:
for name,group in gb_medic:
    idle_times[name] = []
   
    sub_gb = group.groupby('shift_num')
    for name2,group2 in sub_gb:
        idle_total = 0
        time_list = sorted(group2['iiu_tdispatch_id'].append(group2['iiu_tclear_id']), key=lambda x: x[0])
        init = time_list[0][0]
        init = datetime.datetime(init.year, init.month, init.day, 7, 0, 0)
        time_list = [(init, 'c')] + time_list
        
        for entry in time_list:
            if entry[1] == 'c':
                clear_time = entry[0]
            else:
                idle_total += (entry[0] - clear_time).seconds/3600.0
    
        idle_times[name].append(idle_total)

In [ ]:
median_dict = {}
for key in idle_times.keys():
    median_dict[key] = np.median(idle_times[key])
median_dict = sorted(median_dict.items(), key=operator.itemgetter(1))

In [ ]:
median_dict

In [ ]:
median_dict.reverse()

In [ ]:
plt.bar(range(12), [i[1] for i in median_dict][-13:-1],  align='center',color = 'green')
plt.xticks(range(12), [i[0] for i in median_dict][-13:-1] , rotation = 'vertical')
plt.xlim(-0.5,11.5)
plt.title("Median Idle Hours per Shift by Ambulance")
plt.xlabel("Ambulance")
plt.ylabel('Idle Hours')

legend = plt.legend(loc='upper right', shadow=True, fontsize='large')

plt.gcf().subplots_adjust(bottom=0.25)
fig = mpl.pyplot.gcf()
mpl.rcdefaults()

plt.savefig('MedicDemand.pdf')

In [ ]:
rolling_avg_period = 7
rm_idle_times_m46 = running_mean(idle_times['M46'], rolling_avg_period)[:-7]
rm_idle_times_m03 = running_mean(idle_times['M03'], rolling_avg_period)[:-7]

In [ ]:
plt.plot(rm_idle_times_m46, color = 'red', label = 'Ambulance 46')
plt.plot(rm_idle_times_m03, color = 'blue', label = 'Ambulance 03')

plt.title('7 Day Rolling Mean of Idle Hours by Ambulance')
plt.xlabel('Day of Year')
plt.ylabel('Idle Hours')

mpl.rcdefaults()

legend = plt.legend(loc='upper right', shadow=True, fontsize='large')

fig = mpl.pyplot.gcf()
#mpl.rcParams.update({'font.size': 16})
fig.set_size_inches(12,7)

plt.savefig('RollingMean.pdf')

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(rm_idle_times_46.s, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(rm_idle_times_46, lags=40, ax=ax2)

In [ ]:
rm_idle_times_m46.values.sqeeze()

In [ ]:
dta = sm.datasets.sunspots.load_pandas().data

In [ ]:
dta.values.sqeeze

In [ ]:
pacf = stattools.pacf(rm_idle_times_m46)
plt.bar(range(len(pacf)),pacf,  align='center' )
plt.xlim(-0.5,len(pacf)-0.5)

# Busy Medic Analysis

In [ ]:
medic_dict = {}
for unit in year_medic_df['pun_unitid']:
    if len(unit) < 4:
        medic_dict[unit] = 0

In [ ]:
date_of_interest = datetime.datetime(2015,10,12,7,0,0)
shift_of_interest = date_of_interest.timetuple().tm_yday
day_medic_df = year_medic_df[year_medic_df['shift_num'] == shift_of_interest]

In [ ]:
s = sorted(day_medic_df['iiu_tdispatch_id'].append(day_medic_df['iiu_tclear_id']), key = lambda x:x[0])

In [ ]:
all_disp = []
al_one_clear = []
load_tuples = []

In [ ]:
for item in s:
    medic_dict[item[2]] = int(item[1] == 'd')
    if sum(medic_dict.values()) == len(medic_dict):
        all_disp.append(item[0])
    if len(all_disp) > len(al_one_clear):
        if sum(medic_dict.values()) < len(medic_dict):
            al_one_clear.append(item[0])
    load_tuples.append(((item[0]-date_of_interest).days*86400 + (item[0]-date_of_interest).seconds, sum(medic_dict.values())))

In [ ]:
diff_all = [(i-j).seconds for i,j in zip(al_one_clear, all_disp)]
starts_all = [(i-date_of_interest).seconds for i in all_disp]

In [ ]:
gb_medic = day_medic_df.groupby('pun_unitid')

In [ ]:
i = 0

current_palette = sns.color_palette("hls", len(medic_dict))
sns.set(rc={"figure.figsize": (6, 6)})
np.random.seed(sum(map(ord, "palettes")))

dict_range = {}
col_range = {}
for name,group in gb_medic:
    dict_range[name] = (i,1)
    col_range[name] = current_palette[i]
    i += 1

In [ ]:
special_medics = ['M23', 'M29']

ax = plt.subplot(111)

mpl.rcdefaults()

mpl.rcParams['font.family'] = 'monospace'

times = ["7AM", '9AM', '11AM', '1PM', '3PM', '5PM', '7PM', '9PM', '11PM', '1AM', '3AM', '5AM', '7AM']

#ax.set_title("Medic Schedule for " + str(date_of_interest.month) + '/' + str(date_of_interest.day) + '/' + str(date_of_interest.year), fontsize = 20)
ax.set_ylim(0,len(medic_dict))
ax.set_xlim(0, 86400)

ax.set_yticks(np.arange(0.5,12.5,1))

ax.set_xticklabels(times, rotation = 90,  fontsize = 14)
ax.set_xticks(np.arange(0,86400, 7200))

#ax.set_xlabel('Time', fontsize = 16)
#ax.set_ylabel('Medic Unit', fontsize = 16)

medic_units = []

for name,group in gb_medic:
    
    colors = []
    
    medic_units.append(name.replace('M','A'))
    disp = []
    clear = []
    s = sorted(group['iiu_tdispatch_id'].append(group['iiu_tclear_id']), key = lambda x: x[0])
    
    for item in s:
        if item[1] == 'd':
            disp.append(item[0])
        else:
            clear.append(item[0])
            if item[3] == 1:
                colors.append('grey')
            else:
                colors.append('grey')
      
    diff = [(i - j).seconds for i,j in zip(clear,disp)]
    starts = [(i-date_of_interest).seconds for i in disp]
    
    zipped_all = zip(starts_all, diff_all)
    
    zipped = zip(starts, diff)
    
    ax.broken_barh(zipped, dict_range[name], facecolors=colors)

    
    #if name == 'M29':
     #   ax.broken_barh(zipped, dict_range[name], facecolors='blue')
    #elif name == 'M23':
    #    ax.broken_barh(zipped, dict_range[name], facecolors='red')
    #else:
     #   ax.broken_barh(zipped, dict_range[name], facecolors='grey')
    
    ax.broken_barh(zipped_all, dict_range[name], facecolors='black')
    


    
ax.set_yticklabels(medic_units, fontsize = 14)
       

fig = mpl.pyplot.gcf()

fig.set_size_inches(18,7)  

#plt.rc('font', family='sans-serif')

plt.savefig('MedicSched_.pdf')
    
    

In [ ]:
ax = plt.subplot(111)
times = ['7AM', '9AM', '11AM', '1PM', '3PM', '5PM', '7PM', '9PM', '11PM', '1AM', '3AM', '5AM', '7AM']

mpl.rcdefaults()

#.set_title("Busy Medics on " + str(date_of_interest.month) + '/' + str(date_of_interest.day) + '/' + str(date_of_interest.year), fontsize = 20)
ax.set_ylim(0,12)
ax.set_xlim(0, 86400)

ax.set_xticklabels(times, rotation = 90,  fontsize = 18)
ax.set_xticks(np.arange(0,86400, 7200))

#ax.set_xlabel('Time', fontsize = 16)
ax.set_ylabel('Number Ambulances Busy', fontsize = 18)

fig = mpl.pyplot.gcf()

fig.set_size_inches(18,7)

mpl.rcParams['font.family'] = 'monospace'
    
ax.set_yticklabels(np.arange(0,14,2), fontsize = 18)

colors = []

for item in load_tuples:
    if item[1] <= 4:
        colors.append('green')
    elif item[1] <= 8:
        colors.append('yellow')
    else:
        colors.append('red')

col_zip = zip(load_tuples, colors)

s = sorted(col_zip, key = lambda x: x[1])

greens = []
yellows = []
reds = []
for item in s:
    if item[1] == 'red':
        reds.append(item)
    elif item[1] == 'yellow':
        yellows.append(item)
    else:
        greens.append(item)
        
#ax.step([i[0][0] for i in greens], [i[0][1] for i in greens], color = 'green')
#ax.step([i[0][0] for i in yellows], [i[0][1] for i in yellows], color = 'yellow')
#ax.step([i[0][0] for i in reds], [i[0][1] for i in reds], color = 'red')


ax.step([i[0] for i in load_tuples], [i[1] for i in load_tuples], color = 'blue')



plt.savefig('busymedics.pdf')

In [ ]:
groups[30]

In [ ]:
hi_time = 0
lo_time = 0
med_time = 0

num_medics = 0
curr = 0
for item in load_tuples:
    num_medics = item[1]
    
    if num_medics <= 4:
        lo_time +=  item[0] - curr
    elif num_medics <= 8:
        med_time +=  item[0] - curr
    else:
        hi_time +=  item[0] - curr
    curr = item[0]

In [ ]:
print "lo_pct", float(lo_time)/load_tuples[-1][0]
print "med_pct", float(med_time)/load_tuples[-1][0]
print "hi_pct", float(hi_time)/load_tuples[-1][0]

# TESTS

In [ ]:
inc_df = pd.read_sql_query("SELECT i_eventnumber FROM karen.for_ritvik2", conn)

In [ ]:
sub_df = cad_df
sub_df['iiu_tdispatch'] = cad_df['iiu_tdispatch'].apply(dispatch_tag)
sub_df['iiu_tclear'] = cad_df['iiu_tclear'].apply(clear_tag)

In [ ]:
sub_df = sub_df.sort('iiu_tdispatch')

In [ ]:
zip_disp = zip(sub_df['i_eventnumber'], sub_df['iiu_tdispatch'], sub_df['full_unit_id'])
zip_clear = zip(sub_df['i_eventnumber'], sub_df['iiu_tclear'], sub_df['full_unit_id'])
zipped = zip_disp + zip_clear

In [ ]:
s = sorted(zipped, key = lambda x: x[1][0])

In [ ]:
s

In [ ]:
index = 0
for i in range(len(s)):
    if s[i][1][0].hour == 7:
        index = i
        break

In [ ]:
index

In [ ]:
s = s[index:]

In [ ]:
medics = medic_dict.keys()
nums = [int(i[1:]) for i in medics]

In [ ]:
catch = []
for item in s:
    if ((item[2][0] == 'A' and len(item[2]) == 3) or (item[2][0] == 'R' and len(item[2]) == 3)) \
    and (int(item[2].strip('ABCDEFGHIJKLMNOPQRSTUVWXYZ')) in nums) :
        catch.append(item[2])

In [ ]:
busy_dict = {}
for item in catch:
    busy_dict[item] = 0
for item in medic_units:
    busy_dict[item] = 0

In [ ]:
s[:5]

In [ ]:
trouble_inc = ['dummy']
for item in s:
    if (sum(busy_dict.values()) >= 12) and (item[0] != trouble_inc[-1]):
        trouble_inc.append(item[0])
    if item[2] in busy_dict.keys():
        busy_dict[item[2]] = int(item[1][1] == 'd')
    
    
    

In [ ]:
len(trouble_inc)

In [ ]:
trouble_inc = trouble_inc[1:]

In [ ]:
trouble_inc

In [ ]:
interest_ids = inc_df['i_eventnumber']

In [ ]:
my_ids = [i[0] for i in s]

In [ ]:
tf_list = []
for item in interest_ids:
    if item in trouble_inc:
        tf_list.append(False)
    else:
        tf_list.append(True)

In [ ]:
inc_df['medic_available'] = tf_list

In [ ]:
inc_df.to_sql('available_medics2', con = conn_sqlalch, schema='karen')

In [ ]:
count = 0
for i in range(len(tf_list)):
    if tf_list[i] == False:
        count += 1

In [ ]:
count

# Histogram of Times

In [ ]:
time_seq_df = pd.read_sql("""select extract(epoch from aprts.iiu_tclear - incident.i_ttimecreate)/60 as dt
from  clean_csvs.dbo_iincident incident
left join karen.features_table features using (i_eventnumber)
left  join clean_csvs.dbo_rfirehouseapparatus aprts using (i_eventnumber)
where features.trns_to_hosp is true
and (aprts.pun_unitid like 'M__' 
or aprts.pun_unitid like 'A__' 
or aprts.pun_unitid like 'R__')""",conn)

In [ ]:
times = time_seq_df['dt']

mpl.rcdefaults()

iqr = np.subtract(*np.percentile(time_seq_df['dt'], [75, 25]))
max_lim = iqr + np.percentile(time_seq_df['dt'], 75)
min_lim = 0

times_lim = [i for i in times if i >= min_lim and i <= max_lim]

n, bins, patches = plt.hist(times_lim, bins = 100,  color ='red')

plt.xlim(min_lim, max_lim)
plt.xlabel("Minutes", fontsize = 20)
plt.ylabel("Number of Incidents", fontsize = 20)
plt.xticks(np.arange(int(min_lim), int(max_lim)+1,5), fontsize = 15)
plt.xticks(np.arange(0, 5500, fontsize = 15)

plt.axvline(times.mean(), color='b', linestyle='dashed', linewidth=5)

fig = mpl.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)

plt.savefig('HistOfInc.pdf')

